In [2]:
import pandas as pd

In [5]:
twitter_dataset_onlyurl = pd.read_csv('./datasets/yt_onlyurl.csv')

In [6]:
twitter_dataset_onlyurl.columns

Index(['tweetid', 'urls_list', 'url'], dtype='object')

In [5]:
cols = ['tweetid','userid','screen_name','date','lang','location','description','place_id','place_url','place_type','place_name',
        'place_full_name','place_country_code'',''place_country','place_bounding_box','text','extended','coord','reply_userid'
        ,'reply_screen','reply_statusid','tweet_type','friends_count','listed_count','followers_count','favourites_count',
        'statuses_count','verified','hashtag','urls_list','profile_pic_url','profile_banner_url','display_name','date_first_tweet'
        ,'account_creation_date','rt_urls_list','mentionid','mentionsn','rt_screen','rt_userid','rt_user_description','rt_text'
        ,'rt_hashtag','rt_qtd_count','rt_rt_count','rt_reply_count','rt_fav_count','rt_tweetid','rt_location','qtd_screen',
        'qtd_userid','qtd_user_description','qtd_text','qtd_hashtag','qtd_qtd_count','qtd_rt_count','qtd_reply_count',
        'qtd_fav_count','qtd_tweetid','qtd_urls_list','qtd_location','sent_vader',
        'token','media_urls','rt_media_urls','q_media_urls']
print(len(cols))
twitter_dataset = pd.read_csv('./datasets/yturl_twitter.csv',lineterminator="\n",low_memory=False)

65


In [6]:
twitter_dataset

,tweetid,userid,screen_name,date,lang,location,description,place_id,place_url,place_type,...,state,country,rt_state,rt_country,qtd_state,qtd_country,norm_country,norm_rt_country,norm_qtd_country,acc_age
0,1495977545967317000,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:35 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
1,1495977557505753089,2821163219,Billington_Book,Tue Feb 22 04:23:38 +0000 2022,en,"Reus, Tarragonna, Spain","Disabled Vietnam vet, dad, granddad, cancer su...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2691
2,1495977563197476865,294162188,ArsenicMarko,Tue Feb 22 04:23:39 +0000 2022,en,"Aurora, Ontario",NaN,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,3944
3,1495977582155685895,1481149613373538309,MattisseComedy,Tue Feb 22 04:23:44 +0000 2022,en,NaN,Up & Coming Welsh Comedian who isn't afraid t...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40
4,1495977589151830017,1378410853314203649,Denise07190935,Tue Feb 22 04:23:45 +0000 2022,nl,NaN,🧠zelfnadenken \n🕊free Humanity \n📚boeken \nexi...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302697,1509017366180634626,1331144263657287680,Marek82085289,Wed Mar 30 03:59:10 +0000 2022,en,NaN,PO = KORUPCJA i ZDRAJCY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,490
1302698,1509017394299260931,1045050364158853120,life_canbeshort,Wed Mar 30 03:59:17 +0000 2022,en,Canada,The Universe Corrects All Wrongs✌️ pls dont sp...,NaN,NaN,NaN,...,NaN,Canada,NaN,NaN,NaN,NaN,Canada,NaN,NaN,1280
1302699,1509017428831145984,53870131,gusbayu,Wed Mar 30 03:59:25 +0000 2022,in,dipermukaan bumi,Ragadi musuh mepareng \nRihati ya tongwanya ta...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4650
1302700,1509017469758832640,777674879701254144,Online_SergioJu,Wed Mar 30 03:59:35 +0000 2022,es,"Santiago, Chile",Saludos ! aquí verás él Mundo a través de mis...,NaN,NaN,NaN,...,NaN,Chile,NaN,NaN,NaN,NaN,Chile,NaN,NaN,2018


In [22]:
twitter_dataset.columns

Index(['tweetid', 'userid', 'screen_name', 'date', 'lang', 'location',
       'description', 'place_id', 'place_url', 'place_type', 'place_name',
       'place_full_name', 'place_country_code', 'place_country',
       'place_bounding_box', 'text', 'extended', 'coord', 'reply_userid',
       'reply_screen', 'reply_statusid', 'tweet_type', 'friends_count',
       'listed_count', 'followers_count', 'favourites_count', 'statuses_count',
       'verified', 'hashtag', 'urls_list', 'profile_pic_url',
       'profile_banner_url', 'display_name', 'date_first_tweet',
       'account_creation_date', 'rt_urls_list', 'mentionid', 'mentionsn',
       'rt_screen', 'rt_userid', 'rt_user_description', 'rt_text',
       'rt_hashtag', 'rt_qtd_count', 'rt_rt_count', 'rt_reply_count',
       'rt_fav_count', 'rt_tweetid', 'rt_location', 'qtd_screen', 'qtd_userid',
       'qtd_user_description', 'qtd_text', 'qtd_hashtag', 'qtd_qtd_count',
       'qtd_rt_count', 'qtd_reply_count', 'qtd_fav_count', 'qtd_tweeti

In [26]:
print(twitter_dataset['tweetid'].isna().sum())
print(twitter_dataset['userid'].isna().sum())
print(twitter_dataset['screen_name'].isna().sum())
print(twitter_dataset['date'].isna().sum())
#print(twitter_dataset['urls_list'].isna().sum())
#print(twitter_dataset['url_id'].isna().sum())
#print(twitter_dataset['url_type'].isna().sum())
#print(tweetid, userid, screen_name, date, url, url_id, url_type)

0
0
0
0


In [20]:
print('Date format:',twitter_dataset['date'][0].strip())
month_dict = {}
for idx in twitter_dataset.index:
    temp = twitter_dataset['date'][idx].split(' ')[1]
    if temp not in month_dict:
        month_dict[temp] = 1
print(month_dict)

Date format: Tue Feb 22 04:23:35 +0000 2022
{'Feb': 1, 'Mar': 1, 'Apr': 1}


In [28]:
month_dict = {'01': 'Jan','02': 'Feb','03': 'Mar','04': 'Apr',
             '05': 'May','06': 'Jun','07': 'Jul','08': 'Aug',
             '09': 'Sep','10': 'Oct','11': 'Nov','12': 'Dec'}
date_query = input('Input date in MM/DD/YYYY format').split('/')
month_query = month_dict[date_query[0]]
day_query = date_query[1]
year_query = date_query[2]
print(month_query,day_query,year_query)

Input date in MM/DD/YYYY format02/22/2022
Feb 22 2022


In [ ]:
#Extracting rows which fall on the date queried

In [29]:
tweetid_list = []
userid_list = []
screenname_list = []
date_list = []
url_id_list = []
url_type_list = []
#tweetid, userid, screen_name, date, url, url_id, url_type
for idx in twitter_dataset.index:
    date_from_dataset = twitter_dataset['date'][idx].split(' ')
    month_from_dataset = date_from_dataset[1]
    day_from_dataset = date_from_dataset[2]
    year_from_dataset = date_from_dataset[5]
    if month_from_dataset == month_query and day_from_dataset == day_query and year_from_dataset == year_query:
        tweetid_list.append(twitter_dataset['tweetid'][idx])
        userid_list.append(twitter_dataset['userid'][idx])
        screenname_list.append(twitter_dataset['screen_name'][idx])
        date_list.append(twitter_dataset['date'][idx])


['Tue Feb 22 04:23:35 +0000 2022', 'Tue Feb 22 04:23:38 +0000 2022', 'Tue Feb 22 04:23:39 +0000 2022', 'Tue Feb 22 04:23:44 +0000 2022', 'Tue Feb 22 04:23:45 +0000 2022', 'Tue Feb 22 04:23:47 +0000 2022', 'Tue Feb 22 04:23:48 +0000 2022', 'Tue Feb 22 04:23:55 +0000 2022', 'Tue Feb 22 04:23:57 +0000 2022', 'Tue Feb 22 04:24:18 +0000 2022', 'Tue Feb 22 04:24:20 +0000 2022', 'Tue Feb 22 04:24:27 +0000 2022', 'Tue Feb 22 04:24:27 +0000 2022', 'Tue Feb 22 04:24:36 +0000 2022', 'Tue Feb 22 04:24:44 +0000 2022', 'Tue Feb 22 04:24:46 +0000 2022', 'Tue Feb 22 04:24:47 +0000 2022', 'Tue Feb 22 04:24:55 +0000 2022', 'Tue Feb 22 04:24:57 +0000 2022', 'Tue Feb 22 04:25:04 +0000 2022', 'Tue Feb 22 04:25:09 +0000 2022', 'Tue Feb 22 04:25:13 +0000 2022', 'Tue Feb 22 04:25:33 +0000 2022', 'Tue Feb 22 04:25:33 +0000 2022', 'Tue Feb 22 04:25:34 +0000 2022', 'Tue Feb 22 04:25:42 +0000 2022', 'Tue Feb 22 04:25:52 +0000 2022', 'Tue Feb 22 04:26:01 +0000 2022', 'Tue Feb 22 04:26:15 +0000 2022', 'Tue Feb 22 0